In [1]:
import pandas as pd
import re
from multiprocessing import Pool
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from itertools import combinations
from Bio import SeqIO

from Bio import AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Align import MultipleSeqAlignment
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
import numpy as np

import warnings

# Ignorar los SettingWithCopyWarning
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

# Identificación de pares

In [2]:
metadata = pd.read_csv('../../Data/metadata_clades.tsv', sep = '\t')

dict_sp = {
    'Human': 'Human',
    'Humano': 'Human'
}
metadata['host'] = metadata['host'].replace(dict_sp)
metadata = metadata[metadata['host']=='Human']

temporal_clusters = ['2020-03-01', '2020-09-01', '2021-01-01', '2021-05-01', '2022-01-01', '2022-06-01', '2022-11-01', '2023-01-01', '2023-07-01', '2023-09-06']
temporal_clusters = pd.to_datetime(temporal_clusters)


metadata['date'] = pd.to_datetime(metadata['date'])

def assign_cluster(date, clusters):
    for i in range(len(clusters)-1):
        if date >= clusters[i] and date < clusters[i+1]:
            return i
    return len(clusters)-1
    
metadata['cluster'] = metadata['date'].apply(lambda date: assign_cluster(date, temporal_clusters))


metadata.head()

/tmp/ipykernel_23639/2339126653.py:1: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('../../Data/metadata_clades.tsv', sep = '\t')


,strain,virus,gisaid_epi_isl,genbank_accession,date,region,country,division,location,region_exposure,...,paper_url,date_submitted,purpose_of_sequencing,año,mes,clade,clade_who,Group,Group_smooth,cluster
12867,hCoV-19/HongKong/VM20002508-2/2020,betacoronavirus,EPI_ISL_414567,?,2020-02-10,Asia,Hong Kong,NaN,NaN,Asia,...,?,2020-03-13,?,2020,2,19A,NaN,"(2020-01-01, 2020-03-01]","(2010-12-05, 2020-03-01]",9
12868,hCoV-19/USA/WA-UW20/2020,NaN,EPI_ISL_414368,?,2020-03-05,America,USA,Washington,NaN,North America,...,?,2020-03-11,?,2020,3,19B,NaN,"(2020-03-01, 2020-06-01]","(2020-03-01, 2020-09-01]",0
12869,hCoV-19/Canada/BC_02421/2020,betacoronavirus,EPI_ISL_415581,?,2020-03-01,America,Canada,British Columbia,NaN,North America,...,?,2020-03-18,?,2020,3,19A,NaN,"(2020-01-01, 2020-03-01]","(2010-12-05, 2020-03-01]",0
12870,hCoV-19/Netherlands/NA_10/2020,betacoronavirus,EPI_ISL_415466,?,2020-03-09,Europa,Netherlands,NaN,NaN,Europe,...,?,2020-03-17,?,2020,3,20C,NaN,"(2020-03-01, 2020-06-01]","(2020-03-01, 2020-09-01]",0
12871,hCoV-19/Netherlands/Rotterdam_1364740/2020,betacoronavirus,EPI_ISL_413584,?,2020-03-03,Europa,Netherlands,Zuid-Holland,Rotterdam,Europe,...,?,2020-03-07,?,2020,3,20B,NaN,"(2020-03-01, 2020-06-01]","(2020-03-01, 2020-09-01]",0


In [3]:
#6,7,8 ??
metadata['cluster'].value_counts()

3    174637
7     66975
0     64717
2     59456
5     34901
6     29101
4     26886
1     16147
8      6870
9      1912
Name: cluster, dtype: int64

In [4]:
len(metadata)

481602

In [8]:
#headers que no se excluyeron del alineamiento. cat fasta_sin_duplicados.fasta | grep '>' > headers.txt

headers = list(pd.read_csv('/home/ferambriz/Projects/SARS-CoV-2/Data/tree_pode/headers.txt', sep = '\t', header = None)[0])
metadata = metadata[(metadata['strain'].isin(headers))]
len(metadata)

285258

In [9]:
metadata = metadata.drop_duplicates(keep='last')
len(metadata)

251804

In [11]:
#6,7,8 ??
metadata[['cluster', 'Group_smooth']].value_counts()

cluster  Group_smooth            
3        (2021-05-01, 2022-01-01]    122301
0        (2020-03-01, 2020-09-01]     57040
2        (2021-01-01, 2021-05-01]     51708
1        (2020-09-01, 2021-01-01]     13049
5        (2022-06-01, 2022-11-01]      3195
9        (2010-12-05, 2020-03-01]      1844
4        (2022-01-01, 2022-06-01]      1841
3        (2021-01-01, 2021-05-01]       264
1        (2020-03-01, 2020-09-01]       205
2        (2020-09-01, 2021-01-01]       165
0        (2010-12-05, 2020-03-01]       118
4        (2021-05-01, 2022-01-01]        60
5        (2022-01-01, 2022-06-01]        14
dtype: int64

In [12]:
metadata[['Group_smooth']].value_counts()

Group_smooth            
(2021-05-01, 2022-01-01]    122361
(2020-03-01, 2020-09-01]     57245
(2021-01-01, 2021-05-01]     51972
(2020-09-01, 2021-01-01]     13214
(2022-06-01, 2022-11-01]      3195
(2010-12-05, 2020-03-01]      1962
(2022-01-01, 2022-06-01]      1855
dtype: int64

In [13]:
df_db = pd.DataFrame()
for i in tqdm(metadata['country'].unique(), desc='Progreso por país'):
    #for j in metadata['cluster'].unique():
    for j in metadata['Group_smooth'].unique():
        metadata_i = metadata[(metadata['country'] == i) & (metadata['Group_smooth'] == j)]
        lst_i = list(metadata_i['strain'])
        
        df_i = pd.DataFrame(lst_i, columns=['ID'])
        df_i['country'] = i; df_i['Group_smooth'] = j
        df_i.to_csv('../../Data/bin_distanciasEuclideanas2/bin_'+ str(i).replace(' ', '') + '_' + str(j) + '.tsv', sep = '\t', index = False)

Progreso por país: 100%|██████████| 167/167 [00:30<00:00,  5.52it/s]


# Identidad

In [5]:
df = pd.read_csv('../../Data/bin_distanciasEuclideanas/bin_Djibouti_4.tsv', sep = '\t')
lst_i = df['ID']

In [6]:
seq_records = SeqIO.to_dict(SeqIO.parse('/home/ferambriz/Projects/SARS-CoV-2/Data/BloqueHuman/ID_Human/fasta_sin_duplicados.fasta', 'fasta'))

In [7]:
seq_records_filtrados = {id: seq_records[id] for id in lst_i if id in seq_records}

In [8]:
seq_records_filtrados

{'hCoV-19/Djibouti/NAMRU3_F352/2022': SeqRecord(seq=Seq('--------------------------------------------------TTGT...---'), id='hCoV-19/Djibouti/NAMRU3_F352/2022', name='hCoV-19/Djibouti/NAMRU3_F352/2022', description='hCoV-19/Djibouti/NAMRU3_F352/2022', dbxrefs=[]),
 'hCoV-19/Djibouti/NAMRU3_F350/2022': SeqRecord(seq=Seq('--------------------------------------------------TTGT...---'), id='hCoV-19/Djibouti/NAMRU3_F350/2022', name='hCoV-19/Djibouti/NAMRU3_F350/2022', description='hCoV-19/Djibouti/NAMRU3_F350/2022', dbxrefs=[]),
 'hCoV-19/Djibouti/NAMRU3_F300/2022': SeqRecord(seq=Seq('--------------------------------------------------TTGT...---'), id='hCoV-19/Djibouti/NAMRU3_F300/2022', name='hCoV-19/Djibouti/NAMRU3_F300/2022', description='hCoV-19/Djibouti/NAMRU3_F300/2022', dbxrefs=[]),
 'hCoV-19/Djibouti/NAMRU3_F254/2022': SeqRecord(seq=Seq('--------------------------------------------------TTGT...---'), id='hCoV-19/Djibouti/NAMRU3_F254/2022', name='hCoV-19/Djibouti/NAMRU3_F254/2022', de

In [9]:
#alignment = AlignIO.read("aligned.fasta", "fasta")
seq_records_list = list(seq_records_filtrados.values())
alignment = MultipleSeqAlignment(seq_records_list)

In [10]:
calculator = DistanceCalculator('identity')
dm = calculator.get_distance(alignment)
dm

DistanceMatrix(names=['hCoV-19/Djibouti/NAMRU3_F352/2022', 'hCoV-19/Djibouti/NAMRU3_F350/2022', 'hCoV-19/Djibouti/NAMRU3_F300/2022', 'hCoV-19/Djibouti/NAMRU3_F254/2022', 'hCoV-19/Djibouti/NAMRU3_F253/2022', 'hCoV-19/Djibouti/NAMRU3_F247/2022', 'hCoV-19/Djibouti/NAMRU3_F237/2022', 'hCoV-19/Djibouti/NAMRU3_F226/2022', 'hCoV-19/Djibouti/NAMRU3_F217/2022', 'hCoV-19/Djibouti/NAMRU3_F210/2022', 'hCoV-19/Djibouti/NAMRU3_F190/2022', 'hCoV-19/Djibouti/NAMRU3_F179/2022', 'hCoV-19/Djibouti/NAMRU3_F159/2022', 'hCoV-19/Djibouti/NAMRU3_F117/2022', 'hCoV-19/Djibouti/NAMRU3_F084/2022', 'hCoV-19/Djibouti/NAMRU3_F035/2022', 'hCoV-19/Djibouti/NAMRU3_E943/2022', 'hCoV-19/Djibouti/NAMRU3_E942/2022', 'hCoV-19/Djibouti/NAMRU3_E934/2022', 'hCoV-19/Djibouti/NAMRU3_E929/2022', 'hCoV-19/Djibouti/NAMRU3_E924/2022', 'hCoV-19/Djibouti/NAMRU3_E917/2022', 'hCoV-19/Djibouti/NAMRU3_E915/2022', 'hCoV-19/Djibouti/NAMRU3_E914/2022', 'hCoV-19/Djibouti/NAMRU3_E912/2022', 'hCoV-19/Djibouti/NAMRU3_E909/2022', 'hCoV-19/Djibout

In [23]:
np.savetxt('/home/ferambriz/matriz_distancia.tsv', dm, delimiter='\t')

In [31]:

df = pd.read_csv('/home/ferambriz/matriz_distancia.tsv', sep = '\t', header = None)
df.columns = dm.names
df.index = dm.names
df

,hCoV-19/Djibouti/NAMRU3_F352/2022,hCoV-19/Djibouti/NAMRU3_F350/2022,hCoV-19/Djibouti/NAMRU3_F300/2022,hCoV-19/Djibouti/NAMRU3_F254/2022,hCoV-19/Djibouti/NAMRU3_F253/2022,hCoV-19/Djibouti/NAMRU3_F247/2022,hCoV-19/Djibouti/NAMRU3_F237/2022,hCoV-19/Djibouti/NAMRU3_F226/2022,hCoV-19/Djibouti/NAMRU3_F217/2022,hCoV-19/Djibouti/NAMRU3_F210/2022,...,hCoV-19/Djibouti/NAMRU3_E876/2022,hCoV-19/Djibouti/NAMRU3_E875/2022,hCoV-19/Djibouti/NAMRU3_E873/2022,hCoV-19/Djibouti/NAMRU3_E861/2022,hCoV-19/Djibouti/NAMRU3_E859/2022,hCoV-19/Djibouti/NAMRU3_E776/2022,hCoV-19/Djibouti/NAMRU3_E632/2022,hCoV-19/Djibouti/NAMRU3_SOTF_EA_36/2022,hCoV-19/Djibouti/NAMRU3_G788/2022,hCoV-19/Djibouti/NAMRU3_G776/2022
hCoV-19/Djibouti/NAMRU3_F352/2022,0.000000,0.000435,0.000435,0.000635,0.000635,0.000903,0.000669,0.001037,0.000602,0.000869,...,0.000669,0.000602,0.000669,0.000167,0.000635,0.000602,0.000635,0.001505,0.000502,0.000602
hCoV-19/Djibouti/NAMRU3_F350/2022,0.000435,0.000000,0.000401,0.000535,0.000535,0.000803,0.000569,0.000936,0.000502,0.000769,...,0.000569,0.000502,0.000569,0.000334,0.000535,0.000569,0.000535,0.001405,0.000401,0.000502
hCoV-19/Djibouti/NAMRU3_F300/2022,0.000435,0.000401,0.000000,0.000602,0.000602,0.000869,0.000635,0.001003,0.000569,0.000836,...,0.000635,0.000569,0.000635,0.000334,0.000602,0.000569,0.000602,0.001471,0.000468,0.000569
hCoV-19/Djibouti/NAMRU3_F254/2022,0.000635,0.000535,0.000602,0.000000,0.000067,0.001003,0.000769,0.000936,0.000033,0.000903,...,0.000769,0.000702,0.000769,0.000535,0.000736,0.000769,0.000067,0.001405,0.000602,0.000033
hCoV-19/Djibouti/NAMRU3_F253/2022,0.000635,0.000535,0.000602,0.000067,0.000000,0.001003,0.000769,0.000936,0.000033,0.000903,...,0.000769,0.000702,0.000769,0.000535,0.000736,0.000769,0.000067,0.001405,0.000602,0.000033
hCoV-19/Djibouti/NAMRU3_F247/2022,0.000903,0.000803,0.000869,0.001003,0.001003,0.000000,0.000635,0.000936,0.000970,0.000836,...,0.000635,0.000569,0.000635,0.000803,0.000602,0.000635,0.001003,0.001873,0.000869,0.000970
hCoV-19/Djibouti/NAMRU3_F237/2022,0.000669,0.000569,0.000635,0.000769,0.000769,0.000635,0.000000,0.000769,0.000736,0.000401,...,0.000134,0.000134,0.000134,0.000569,0.000100,0.000134,0.000769,0.001639,0.000635,0.000736
hCoV-19/Djibouti/NAMRU3_F226/2022,0.001037,0.000936,0.001003,0.000936,0.000936,0.000936,0.000769,0.000000,0.000903,0.000368,...,0.000769,0.000702,0.000769,0.000936,0.000736,0.000769,0.000936,0.001405,0.001003,0.000903
hCoV-19/Djibouti/NAMRU3_F217/2022,0.000602,0.000502,0.000569,0.000033,0.000033,0.000970,0.000736,0.000903,0.000000,0.000869,...,0.000736,0.000669,0.000736,0.000502,0.000702,0.000736,0.000033,0.001371,0.000569,0.000000
hCoV-19/Djibouti/NAMRU3_F210/2022,0.000869,0.000769,0.000836,0.000903,0.000903,0.000836,0.000401,0.000368,0.000869,0.000000,...,0.000401,0.000334,0.000401,0.000769,0.000368,0.000401,0.000903,0.001572,0.000836,0.000869


In [24]:
constructor = DistanceTreeConstructor()
upgmatree = constructor.upgma(dm)
print(upgmatree)

Tree(rooted=True)
    Clade(branch_length=0, name='Inner42')
        Clade(branch_length=7.382267769956202e-05, name='Inner41')
            Clade(branch_length=0.0005768651974718264, name='hCoV-19/Djibouti/NAMRU3_F035/2022')
            Clade(branch_length=0.0002926127813262791, name='Inner36')
                Clade(branch_length=0.0002842524161455473, name='hCoV-19/Djibouti/NAMRU3_SOTF_EA_36/2022')
                Clade(branch_length=0.0002842524161455473, name='Inner12')
                    Clade(branch_length=0.0, name='Inner11')
                        Clade(branch_length=0.0, name='hCoV-19/Djibouti/NAMRU3_F117/2022')
                        Clade(branch_length=0.0, name='hCoV-19/Djibouti/NAMRU3_F179/2022')
                    Clade(branch_length=0.0, name='hCoV-19/Djibouti/NAMRU3_F190/2022')
        Clade(branch_length=0.00026304125526703927, name='Inner40')
            Clade(branch_length=5.3701466773824745e-05, name='Inner39')
                Clade(branch_length=0.00020371858592

# Poda

In [6]:
df = pd.read_csv('../../Data/identity_final/bin_Djibouti_4.tsv', sep = '\t')
df

,Unnamed: 0,hCoV-19/Djibouti/NAMRU3_F352/2022,hCoV-19/Djibouti/NAMRU3_F350/2022,hCoV-19/Djibouti/NAMRU3_F300/2022,hCoV-19/Djibouti/NAMRU3_F254/2022,hCoV-19/Djibouti/NAMRU3_F253/2022,hCoV-19/Djibouti/NAMRU3_F247/2022,hCoV-19/Djibouti/NAMRU3_F237/2022,hCoV-19/Djibouti/NAMRU3_F226/2022,hCoV-19/Djibouti/NAMRU3_F217/2022,...,hCoV-19/Djibouti/NAMRU3_E876/2022,hCoV-19/Djibouti/NAMRU3_E875/2022,hCoV-19/Djibouti/NAMRU3_E873/2022,hCoV-19/Djibouti/NAMRU3_E861/2022,hCoV-19/Djibouti/NAMRU3_E859/2022,hCoV-19/Djibouti/NAMRU3_E776/2022,hCoV-19/Djibouti/NAMRU3_E632/2022,hCoV-19/Djibouti/NAMRU3_SOTF_EA_36/2022,hCoV-19/Djibouti/NAMRU3_G788/2022,hCoV-19/Djibouti/NAMRU3_G776/2022
0,hCoV-19/Djibouti/NAMRU3_F352/2022,0.000000,0.000435,0.000435,0.000635,0.000635,0.000903,0.000669,0.001037,0.000602,...,0.000669,0.000602,0.000669,0.000167,0.000635,0.000602,0.000635,0.001505,0.000502,0.000602
1,hCoV-19/Djibouti/NAMRU3_F350/2022,0.000435,0.000000,0.000401,0.000535,0.000535,0.000803,0.000569,0.000936,0.000502,...,0.000569,0.000502,0.000569,0.000334,0.000535,0.000569,0.000535,0.001405,0.000401,0.000502
2,hCoV-19/Djibouti/NAMRU3_F300/2022,0.000435,0.000401,0.000000,0.000602,0.000602,0.000869,0.000635,0.001003,0.000569,...,0.000635,0.000569,0.000635,0.000334,0.000602,0.000569,0.000602,0.001471,0.000468,0.000569
3,hCoV-19/Djibouti/NAMRU3_F254/2022,0.000635,0.000535,0.000602,0.000000,0.000067,0.001003,0.000769,0.000936,0.000033,...,0.000769,0.000702,0.000769,0.000535,0.000736,0.000769,0.000067,0.001405,0.000602,0.000033
4,hCoV-19/Djibouti/NAMRU3_F253/2022,0.000635,0.000535,0.000602,0.000067,0.000000,0.001003,0.000769,0.000936,0.000033,...,0.000769,0.000702,0.000769,0.000535,0.000736,0.000769,0.000067,0.001405,0.000602,0.000033
5,hCoV-19/Djibouti/NAMRU3_F247/2022,0.000903,0.000803,0.000869,0.001003,0.001003,0.000000,0.000635,0.000936,0.000970,...,0.000635,0.000569,0.000635,0.000803,0.000602,0.000635,0.001003,0.001873,0.000869,0.000970
6,hCoV-19/Djibouti/NAMRU3_F237/2022,0.000669,0.000569,0.000635,0.000769,0.000769,0.000635,0.000000,0.000769,0.000736,...,0.000134,0.000134,0.000134,0.000569,0.000100,0.000134,0.000769,0.001639,0.000635,0.000736
7,hCoV-19/Djibouti/NAMRU3_F226/2022,0.001037,0.000936,0.001003,0.000936,0.000936,0.000936,0.000769,0.000000,0.000903,...,0.000769,0.000702,0.000769,0.000936,0.000736,0.000769,0.000936,0.001405,0.001003,0.000903
8,hCoV-19/Djibouti/NAMRU3_F217/2022,0.000602,0.000502,0.000569,0.000033,0.000033,0.000970,0.000736,0.000903,0.000000,...,0.000736,0.000669,0.000736,0.000502,0.000702,0.000736,0.000033,0.001371,0.000569,0.000000
9,hCoV-19/Djibouti/NAMRU3_F210/2022,0.000869,0.000769,0.000836,0.000903,0.000903,0.000836,0.000401,0.000368,0.000869,...,0.000401,0.000334,0.000401,0.000769,0.000368,0.000401,0.000903,0.001572,0.000836,0.000869


In [68]:
umbral = 0
lst_mantein = []; lst_pode = []

for i in df['Unnamed: 0']:
    print(i)
    
    df_i = df[df['Unnamed: 0'] == i]
    df_i.drop(['Unnamed: 0'], axis = 1, inplace = True)
    df_i = df_i.T

    df_i = df_i[(df_i[df_i.columns[0]] <= umbral) & (~df_i.index.isin([i]))]
    
    if len(df_i) > 0:
        if i not in lst_pode:
            lst_mantein.append(i)
        print(df_i.columns[0])
        for j in df_i.index:
            if j not in lst_mantein:
                lst_pode.append(j)
    
    print(df_i)
    print('=====')

hCoV-19/Djibouti/NAMRU3_F352/2022
Empty DataFrame
Columns: [0]
Index: []
=====
hCoV-19/Djibouti/NAMRU3_F350/2022
1
                                     1
hCoV-19/Djibouti/NAMRU3_E877/2022  0.0
=====
hCoV-19/Djibouti/NAMRU3_F300/2022
Empty DataFrame
Columns: [2]
Index: []
=====
hCoV-19/Djibouti/NAMRU3_F254/2022
Empty DataFrame
Columns: [3]
Index: []
=====
hCoV-19/Djibouti/NAMRU3_F253/2022
Empty DataFrame
Columns: [4]
Index: []
=====
hCoV-19/Djibouti/NAMRU3_F247/2022
Empty DataFrame
Columns: [5]
Index: []
=====
hCoV-19/Djibouti/NAMRU3_F237/2022
Empty DataFrame
Columns: [6]
Index: []
=====
hCoV-19/Djibouti/NAMRU3_F226/2022
Empty DataFrame
Columns: [7]
Index: []
=====
hCoV-19/Djibouti/NAMRU3_F217/2022
8
                                     8
hCoV-19/Djibouti/NAMRU3_E943/2022  0.0
hCoV-19/Djibouti/NAMRU3_E942/2022  0.0
hCoV-19/Djibouti/NAMRU3_E914/2022  0.0
hCoV-19/Djibouti/NAMRU3_E908/2022  0.0
hCoV-19/Djibouti/NAMRU3_E905/2022  0.0
hCoV-19/Djibouti/NAMRU3_E901/2022  0.0
hCoV-19/Djibouti/N

In [73]:
lst_pode = list(set(lst_pode))
lst_pode

['hCoV-19/Djibouti/NAMRU3_E905/2022',
 'hCoV-19/Djibouti/NAMRU3_E942/2022',
 'hCoV-19/Djibouti/NAMRU3_E943/2022',
 'hCoV-19/Djibouti/NAMRU3_E908/2022',
 'hCoV-19/Djibouti/NAMRU3_E912/2022',
 'hCoV-19/Djibouti/NAMRU3_F117/2022',
 'hCoV-19/Djibouti/NAMRU3_G776/2022',
 'hCoV-19/Djibouti/NAMRU3_E897/2022',
 'hCoV-19/Djibouti/NAMRU3_E877/2022',
 'hCoV-19/Djibouti/NAMRU3_F179/2022',
 'hCoV-19/Djibouti/NAMRU3_E914/2022',
 'hCoV-19/Djibouti/NAMRU3_E901/2022']

In [82]:
df[~df['Unnamed: 0'].isin(lst_pode)]['Unnamed: 0'].to_csv('/home/ferambriz/test.tsv', sep='\t', index = False, header = False)